# 🐍 Mastering Object-Oriented Programming (OOP) in Python
**Welcome!** This notebook is your comprehensive guide to understanding and applying Object-Oriented Programming (OOP) principles in Python. We'll explore the core concepts, best practices, and enterprise-level considerations that make OOP a powerful paradigm for building robust, maintainable, and scalable software.
**Target Audience:** Beginners to intermediate Python developers looking to solidify their OOP foundations and understand its application in real-world scenarios.
**Learning Objectives:**
*   Understand the fundamental principles of OOP: Encapsulation, Abstraction, Inheritance, and Polymorphism.
*   Learn how to define classes, create objects, and implement methods (`__init__`, instance methods, `@classmethod`, `@staticmethod`).
*   Grasp the importance of data hiding and access control (Encapsulation).
*   Implement inheritance hierarchies for code reuse and specialization.
*   Leverage polymorphism for flexible and extensible designs.
*   Understand Abstraction using Abstract Base Classes (ABCs).
*   Incorporate enterprise best practices: Logging, Type Hinting, Docstrings, Basic Testing, and Dependency Injection concepts.
*   Identify common pitfalls and prepare for related interview questions.

## 1. Introduction: What is OOP and Why Use It?
**Analogy: The Car Factory**
Imagine a car factory. Instead of building every single car from scratch with unique instructions, the factory uses **blueprints** (Classes) to define the common structure and capabilities of a car (e.g., engine, wheels, doors, accelerate, brake). Each car rolling off the assembly line is an **instance** (Object) built from that blueprint. While all cars share the blueprint's design, each can have unique characteristics (color, VIN number - *attributes*) and perform actions (accelerate, brake - *methods*).
**Core Idea:** OOP is a programming paradigm based on the concept of "objects," which can contain data in the form of fields (often known as attributes or properties) and code in the form of procedures (often known as methods).
**Key Benefits in Enterprise Software:**
1.  **Modularity:** Objects are self-contained units. This makes complex systems easier to manage and understand.
2.  **Reusability:** Inheritance allows classes to reuse code from parent classes, reducing redundancy.
3.  **Maintainability:** Changes within one object/class are less likely to break unrelated parts of the system (if designed well). Encapsulation helps protect internal state.
4.  **Scalability:** Modular design makes it easier to add new features or components without rewriting existing code.
5.  **Collaboration:** Different teams can work on different objects/components relatively independently.

## 2. Classes and Objects: The Blueprints and Instances
*   **Class:** A blueprint or template for creating objects. It defines attributes (data) and methods (functions) that the objects created from it will have.
*   **Object (Instance):** A specific realization created from a class. It has its own state (values for its attributes) and can perform actions defined by the class's methods.

### 2.1 Defining a Simple Class

In [16]:
import logging
from typing import Optional, List, Type, TypeVar

# Configure basic logging for demonstration
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Generic TypeVar for factory method demonstration later
T = TypeVar('T', bound='Vehicle')

class Vehicle:
    """
    Represents a generic vehicle.

    Attributes:
        make (str): The manufacturer of the vehicle.
        model (str): The model of the vehicle.
        year (int): The manufacturing year.
        _mileage (int): The current mileage (intended as protected).
    """
    # Class attribute (shared by all instances)
    num_vehicles_created: int = 0

    def __init__(self, make: str, model: str, year: int, initial_mileage: int = 0):
        """
        Initializes a new Vehicle instance.

        Args:
            make: The manufacturer name.
            model: The vehicle model name.
            year: The manufacturing year.
            initial_mileage: The starting mileage (default 0).

        Raises:
            ValueError: If year or initial_mileage are invalid.
        """
        if year <= 1885: # First car roughly 1886
             raise ValueError("Year must be after 1885.")
        if initial_mileage < 0:
             raise ValueError("Initial mileage cannot be negative.")

        # Instance attributes
        self.make = make
        self.model = model
        self.year = year
        self._mileage = initial_mileage # Convention: single underscore suggests "protected"

        Vehicle.num_vehicles_created += 1
        logging.info(f"Created Vehicle: {self.make} {self.model} ({self.year})")

    def drive(self, distance: int) -> None:
        """
        Simulates driving the vehicle, increasing its mileage.

        Args:
            distance: The distance driven in miles.

        Raises:
            ValueError: If distance is negative.
        """
        if distance < 0:
            raise ValueError("Distance driven cannot be negative.")
        self._mileage += distance
        print(f"{self.make} {self.model} drove {distance} miles. Total mileage: {self._mileage}")

    def get_description(self) -> str:
        """Returns a string description of the vehicle."""
        return f"{self.year} {self.make} {self.model} with {self._mileage} miles."

    # String representation methods
    def __str__(self) -> str:
        """User-friendly string representation."""
        return f"{self.year} {self.make} {self.model}"

    def __repr__(self) -> str:
        """Developer-friendly, unambiguous string representation."""
        return f"Vehicle(make='{self.make}', model='{self.model}', year={self.year}, initial_mileage={self._mileage})"

    # --- Special Methods: @classmethod and @staticmethod ---

    @classmethod
    def get_total_vehicles(cls) -> int:
        """Returns the total number of Vehicle instances created."""
        # cls refers to the class itself (Vehicle)
        return cls.num_vehicles_created

    @classmethod
    def from_string(cls: Type[T], vehicle_str: str) -> T:
        """
        Factory method to create a Vehicle instance from a string.
        Expected format: "Make-Model-Year-Mileage" e.g., "Toyota-Camry-2020-15000"

        Args:
            vehicle_str: The string containing vehicle data.

        Returns:
            A new instance of the class (or subclass) this method is called on.

        Raises:
            ValueError: If the string format is incorrect or data is invalid.
        """
        try:
            make, model, year_str, mileage_str = vehicle_str.split('-')
            year = int(year_str)
            mileage = int(mileage_str)
            # Use cls() to create an instance of the *actual* class this method is called on
            # This allows subclasses to use this factory method correctly!
            logging.info(f"Creating vehicle from string using {cls.__name__}")
            return cls(make=make, model=model, year=year, initial_mileage=mileage)
        except (ValueError, IndexError) as e:
            logging.error(f"Failed to create vehicle from string: '{vehicle_str}'. Error: {e}")
            raise ValueError(f"Invalid vehicle string format or data: '{vehicle_str}'") from e

    @staticmethod
    def is_vintage(year: int) -> bool:
        """
        Checks if a vehicle year is considered vintage (e.g., > 40 years old).
        Static methods don't receive 'self' or 'cls'. They are utility functions
        logically grouped with the class.

        Args:
            year: The year to check.

        Returns:
            True if the year is considered vintage, False otherwise.
        """
        import datetime
        current_year = datetime.datetime.now().year
        return (current_year - year) > 40

### 2.2 Creating Objects (Instances)

In [17]:
# Create instances of the Vehicle class
try:
    car1 = Vehicle("Toyota", "Camry", 2020, 15000)
    car2 = Vehicle(make="Honda", model="Civic", year=2019) # Using keyword arguments
    # car3 = Vehicle("Ford", "Model T", 1880) # This would raise ValueError
except ValueError as e:
    logging.error(f"Error creating vehicle: {e}")

print(f"Car 1: {car1}") # Uses __str__
print(f"Car 2 as object: {car2!r}") # Forces __repr__

2025-04-20 16:13:27,511 - INFO - Created Vehicle: Toyota Camry (2020)
2025-04-20 16:13:27,513 - INFO - Created Vehicle: Honda Civic (2019)


Car 1: 2020 Toyota Camry
Car 2 as object: Vehicle(make='Honda', model='Civic', year=2019, initial_mileage=0)


### 2.3 Accessing Attributes and Calling Methods

In [18]:
print(f"Car 1 Make: {car1.make}")
# print(f"Car 1 Mileage: {car1._mileage}") # Possible, but violates convention (see Encapsulation)

car1.drive(150)
car2.drive(80)

print(car1.get_description())
print(car2.get_description())

Car 1 Make: Toyota
Toyota Camry drove 150 miles. Total mileage: 15150
Honda Civic drove 80 miles. Total mileage: 80
2020 Toyota Camry with 15150 miles.
2019 Honda Civic with 80 miles.


### 2.4 Class Attributes vs Instance Attributes

In [19]:
print(f"Total vehicles created: {Vehicle.num_vehicles_created}")
print(f"Total vehicles via class method: {Vehicle.get_total_vehicles()}")
print(f"Total vehicles via instance: {car1.get_total_vehicles()}") # Can call class methods on instances too

Total vehicles created: 2
Total vehicles via class method: 2
Total vehicles via instance: 2


### 2.5 Using `@classmethod` and `@staticmethod`

In [20]:
# Using the static method
print(f"Is car1 vintage? {Vehicle.is_vintage(car1.year)}")
print(f"Is a 1975 car vintage? {Vehicle.is_vintage(1975)}")

# Using the class method factory
try:
    car_from_str = Vehicle.from_string("Ford-Mustang-2021-5000")
    print(f"Created from string: {car_from_str!r}")
    print(f"Total vehicles now: {Vehicle.get_total_vehicles()}")

    # Invalid string
    # car_invalid = Vehicle.from_string("Tesla-Model3-Oops")
except ValueError as e:
    print(f"Error: {e}")

2025-04-20 16:13:27,551 - INFO - Creating vehicle from string using Vehicle
2025-04-20 16:13:27,554 - INFO - Created Vehicle: Ford Mustang (2021)


Is car1 vintage? False
Is a 1975 car vintage? True
Created from string: Vehicle(make='Ford', model='Mustang', year=2021, initial_mileage=5000)
Total vehicles now: 3


## 3. Encapsulation: Protecting Your Data
**Concept:** Bundling data (attributes) and methods that operate on the data within a single unit (the class). Crucially, it also involves restricting direct access to some of an object's components, which is a key principle for preventing accidental modification of data (data hiding).
**Python's Approach:**
*   **No strict `private` keyword:** Python relies on conventions.
*   **`_` (Single Underscore - Protected):** Convention indicating an attribute or method is intended for internal use or use by subclasses. It's a hint to developers, but *not* enforced by the interpreter.
*   **`__` (Double Underscore - "Private" / Name Mangling):** Attributes or methods starting with `__` (and without trailing `__`) undergo *name mangling*. Python changes the name to `_ClassName__attributeName`. This makes it harder (but not impossible) to access directly from outside the class, primarily to avoid naming conflicts in subclasses.
*   **Properties (`@property`):** The Pythonic way to provide controlled access (getters, setters, deleters) to "private" or "protected" attributes without exposing them directly.
**Real-world Analogy:** Think of a car's dashboard. It *exposes* controls like the steering wheel, accelerator, and speedometer (`public` methods/properties), but it *hides* the complex engine mechanics and wiring (`protected`/`private` implementation details). You interact through the defined interface.

### 3.1 Demonstrating Access Conventions and Properties

In [21]:
class BankAccount:
    """
    Represents a bank account with controlled access to balance.

    Attributes:
        account_holder (str): Name of the account holder.
        __balance (float): Current balance (name-mangled "private").
    """
    MIN_BALANCE = 10.0 # A constant for minimum balance

    def __init__(self, account_holder: str, initial_deposit: float):
        """
        Initializes a BankAccount.

        Args:
            account_holder: The name of the account holder.
            initial_deposit: The starting balance.

        Raises:
            ValueError: If initial deposit is below minimum allowed.
        """
        self.account_holder = account_holder # Public attribute
        if initial_deposit < self.MIN_BALANCE:
            raise ValueError(f"Initial deposit must be at least {self.MIN_BALANCE}")
        # Name Mangling: This becomes _BankAccount__balance internally
        self.__balance = initial_deposit
        logging.info(f"Account created for {self.account_holder}")

    # Getter using @property decorator
    @property
    def balance(self) -> float:
        """Gets the current account balance."""
        # Could add access logging or checks here if needed
        logging.debug(f"Accessed balance for {self.account_holder}")
        return self.__balance

    # Setter using @balance.setter decorator
    @balance.setter
    def balance(self, new_balance: float) -> None:
        """Sets the balance (rarely used directly, prefer deposit/withdraw)."""
        print("Warning: Directly setting balance is generally discouraged. Use deposit/withdraw.")
        if new_balance < 0:
             raise ValueError("Balance cannot be negative.")
        # You *could* allow direct setting, but often methods are better
        # self.__balance = new_balance
        logging.warning(f"Balance directly set for {self.account_holder} to {new_balance}")
        # Let's prevent direct setting for this example
        raise AttributeError("Balance cannot be set directly. Use deposit() or withdraw().")


    # Deleter using @balance.deleter decorator (less common)
    @balance.deleter
    def balance(self) -> None:
        """Deletes the balance attribute (not recommended for balance!)."""
        print("Deleting balance attribute is not allowed.")
        # del self.__balance # Avoid doing this!
        raise AttributeError("Cannot delete the balance attribute.")

    def deposit(self, amount: float) -> None:
        """Deposits funds into the account."""
        if amount <= 0:
            raise ValueError("Deposit amount must be positive.")
        self.__balance += amount
        logging.info(f"Deposited {amount}. New balance: {self.__balance}")
        print(f"Deposited ${amount:.2f}. New balance: ${self.__balance:.2f}")

    def withdraw(self, amount: float) -> None:
        """Withdraws funds from the account."""
        if amount <= 0:
            raise ValueError("Withdrawal amount must be positive.")
        if self.__balance - amount < self.MIN_BALANCE:
            raise ValueError(f"Insufficient funds. Minimum balance of {self.MIN_BALANCE} required.")
        self.__balance -= amount
        logging.info(f"Withdrew {amount}. New balance: {self.__balance}")
        print(f"Withdrew ${amount:.2f}. New balance: ${self.__balance:.2f}")

    def __str__(self) -> str:
        return f"Account Holder: {self.account_holder}" # Don't usually show balance in __str__

    def __repr__(self) -> str:
        # Careful about showing sensitive info like balance in repr in real apps
        return f"BankAccount(account_holder='{self.account_holder}', initial_deposit=***)"

In [22]:
# Working with the BankAccount
try:
    acc1 = BankAccount("Alice Smith", 1000.0)
    print(acc1)
    print(repr(acc1))

    # Accessing balance via the property (getter)
    print(f"Initial Balance: ${acc1.balance:.2f}")

    acc1.deposit(500.50)
    acc1.withdraw(200.0)

    print(f"Final Balance: ${acc1.balance:.2f}")

    # Trying to access the mangled name directly (possible but bad practice)
    print(f"Accessing mangled balance: {acc1._BankAccount__balance}") # Don't do this!

    # Trying to set the balance directly using the property (will fail due to our setter)
    try:
        acc1.balance = 5000.0
    except AttributeError as e:
        print(f"Error setting balance directly: {e}")

    # Trying to set the mangled attribute directly (bypasses setter logic!)
    acc1._BankAccount__balance = 10.0 # Bypasses checks! Very bad practice.
    print(f"Balance after unsafe direct modification: ${acc1.balance:.2f}")

    # Trying to withdraw below minimum
    try:
        acc1.withdraw(5.0) # Will fail as 10 - 5 < 10
    except ValueError as e:
        print(f"Error withdrawing: {e}")

except ValueError as e:
    logging.error(f"Error creating/using BankAccount: {e}")

2025-04-20 16:13:27,593 - INFO - Account created for Alice Smith
2025-04-20 16:13:27,594 - INFO - Deposited 500.5. New balance: 1500.5
2025-04-20 16:13:27,596 - INFO - Withdrew 200.0. New balance: 1300.5
2025-04-20 16:13:27,597 - WARNING - Balance directly set for Alice Smith to 5000.0


Account Holder: Alice Smith
BankAccount(account_holder='Alice Smith', initial_deposit=***)
Initial Balance: $1000.00
Deposited $500.50. New balance: $1500.50
Withdrew $200.00. New balance: $1300.50
Final Balance: $1300.50
Accessing mangled balance: 1300.5
Error setting balance directly: Balance cannot be set directly. Use deposit() or withdraw().
Balance after unsafe direct modification: $10.00
Error withdrawing: Insufficient funds. Minimum balance of 10.0 required.


**Pitfall:** Relying solely on `_` or `__` for security. They are primarily conventions and name mangling mechanisms, not true access restrictions like in Java or C++. Someone *can* still access `_protected` and `_ClassName__private` attributes if they try. Properties provide better control.

## 4. Inheritance: Standing on the Shoulders of Giants
**Concept:** A mechanism where a new class (subclass or derived class) inherits attributes and methods from an existing class (superclass or base class). This promotes code reuse and establishes an "IS-A" relationship (e.g., a `Car` IS-A `Vehicle`).
**Key Features:**
*   **Code Reuse:** Subclasses automatically get the base class's functionality.
*   **Specialization:** Subclasses can add their own unique attributes and methods.
*   **Method Overriding:** Subclasses can provide a specific implementation for a method already defined in the base class.
*   **`super()`:** A function used in the subclass to call methods of the superclass, especially useful for extending `__init__` or overridden methods.
**Real-world Analogy:** Think of biological classification. A `Dog` IS-A `Mammal`. It inherits characteristics of mammals (warm-blooded, hair) but adds its own specific traits (barking, breeds).

### 4.1 Defining Subclasses

In [23]:
class ElectricCar(Vehicle): # Inherits from Vehicle
    """
    Represents an electric car, inheriting from Vehicle.

    Attributes:
        battery_kwh (float): The capacity of the battery in kilowatt-hours.
        _charge_level (float): The current battery charge level (0.0 to 1.0).
    """

    def __init__(self, make: str, model: str, year: int, battery_kwh: float, initial_mileage: int = 0):
        """
        Initializes an ElectricCar instance.

        Args:
            make: Manufacturer name.
            model: Model name.
            year: Manufacturing year.
            battery_kwh: Battery capacity in kWh.
            initial_mileage: Starting mileage (default 0).
        """
        # Call the parent class's __init__ method
        super().__init__(make=make, model=model, year=year, initial_mileage=initial_mileage)
        if battery_kwh <= 0:
            raise ValueError("Battery kWh must be positive.")
        self.battery_kwh = battery_kwh
        self._charge_level = 1.0 # Start fully charged by default
        logging.info(f"Created ElectricCar: {self.make} {self.model} with {self.battery_kwh}kWh battery")

    # --- Method Overriding ---
    def get_description(self) -> str:
        """Overrides the parent method to include battery info."""
        # Call the parent's method first, then add specific info
        base_description = super().get_description()
        return f"{base_description}, Battery: {self.battery_kwh}kWh, Charge: {self._charge_level*100:.1f}%"

    # --- New methods specific to ElectricCar ---
    def charge(self, charge_percent: float) -> None:
        """Charges the battery."""
        if not 0.0 <= charge_percent <= 100.0:
             raise ValueError("Charge percentage must be between 0 and 100.")
        charge_amount = charge_percent / 100.0
        self._charge_level = min(1.0, self._charge_level + charge_amount) # Cap at 1.0
        print(f"{self.make} {self.model} charged. Current level: {self._charge_level*100:.1f}%")

    def drive(self, distance: int) -> None:
        """
        Overrides drive to simulate battery consumption.
        (Simplified consumption calculation).
        """
        if self._charge_level <= 0:
            print(f"{self.make} {self.model} battery is empty! Cannot drive.")
            return

        # Simple simulation: assume 5 miles per kWh, 100% efficiency
        # And let's say driving uses 1% charge per 3 miles for this example
        estimated_charge_needed = (distance / 3.0) / 100.0

        if estimated_charge_needed > self._charge_level:
            max_distance = int(self._charge_level * 100 * 3)
            print(f"Not enough charge for {distance} miles. Can drive approx {max_distance} miles.")
            # Drive the max possible distance
            super().drive(max_distance)
            self._charge_level = 0.0
        else:
             # Call parent's drive method first to update mileage
            super().drive(distance)
             # Then update charge level
            self._charge_level -= estimated_charge_needed
            print(f"Battery level after driving: {self._charge_level*100:.1f}%")

    # --- Using the @classmethod factory from the parent ---
    # No need to redefine from_string if the parent's works, but let's show
    # how a subclass *could* tailor it if needed (though often not necessary)
    @classmethod
    def from_string(cls: Type[T], vehicle_str: str, battery_kwh: Optional[float] = None) -> T:
        """
        Factory method specifically for ElectricCar, extending the base.
        Expected format: "Make-Model-Year-Mileage"
        Requires battery_kwh as a separate argument for EC.
        """
        if battery_kwh is None:
             raise ValueError("battery_kwh is required for ElectricCar.from_string")
        # We could parse battery_kwh from the string too if we changed format

        # Call the *parent's* classmethod to handle the common parts
        # Note: super().from_string(...) won't work directly like this inside @classmethod
        # because super() needs an instance or class context passed properly.
        # It's often easier to just parse again or call the base __init__.

        try:
            make, model, year_str, mileage_str = vehicle_str.split('-')
            year = int(year_str)
            mileage = int(mileage_str)
            logging.info(f"Creating ElectricCar from string using {cls.__name__}")
            # Call cls() which correctly points to ElectricCar.__init__
            return cls(make=make, model=model, year=year, battery_kwh=battery_kwh, initial_mileage=mileage)
        except (ValueError, IndexError) as e:
            logging.error(f"Failed to create ElectricCar from string: '{vehicle_str}'. Error: {e}")
            raise ValueError(f"Invalid ElectricCar string format or data: '{vehicle_str}'") from e

    def __repr__(self) -> str:
        """Developer-friendly representation for ElectricCar."""
        # Note: Accessing _mileage directly here as it's conventional within the class hierarchy
        return (f"ElectricCar(make='{self.make}', model='{self.model}', year={self.year}, "
                f"battery_kwh={self.battery_kwh}, initial_mileage={self._mileage})")


class Truck(Vehicle): # Another subclass
    """Represents a truck with towing capacity."""
    def __init__(self, make: str, model: str, year: int, towing_capacity_lbs: int, initial_mileage: int = 0):
        super().__init__(make, model, year, initial_mileage)
        if towing_capacity_lbs < 0:
            raise ValueError("Towing capacity cannot be negative.")
        self.towing_capacity_lbs = towing_capacity_lbs
        logging.info(f"Created Truck: {self.make} {self.model} with {self.towing_capacity_lbs}lbs towing")

    def get_description(self) -> str:
        """Overrides description to include towing capacity."""
        return f"{super().get_description()}, Towing Capacity: {self.towing_capacity_lbs} lbs"

    def tow(self, weight_lbs: int):
        """Simulates towing."""
        if weight_lbs <= 0:
            print("Weight to tow must be positive.")
        elif weight_lbs <= self.towing_capacity_lbs:
            print(f"{self.make} {self.model} is towing {weight_lbs} lbs.")
        else:
            print(f"Cannot tow {weight_lbs} lbs. Exceeds capacity of {self.towing_capacity_lbs} lbs.")

    def __repr__(self) -> str:
        return (f"Truck(make='{self.make}', model='{self.model}', year={self.year}, "
                f"towing_capacity_lbs={self.towing_capacity_lbs}, initial_mileage={self._mileage})")

In [24]:
# Creating instances of subclasses
try:
    my_tesla = ElectricCar("Tesla", "Model 3", 2022, 75.0, 5000)
    my_f150 = Truck("Ford", "F-150", 2021, 10000, 12000)

    print(my_tesla.get_description())
    print(my_f150.get_description())

    my_tesla.drive(100)
    my_tesla.charge(20)
    my_f150.tow(8000)
    my_f150.tow(12000) # Exceeds capacity

    # Demonstrate drive override with insufficient charge
    my_tesla.drive(5000) # Will drain battery

    # Using the tailored class method
    ec_from_str = ElectricCar.from_string("Nissan-Leaf-2019-22000", battery_kwh=40.0)
    print(f"Electric Car from string: {ec_from_str!r}")
    print(ec_from_str.get_description())

    # Demonstrate inheritance of class attributes/methods
    print(f"\nTotal vehicles (incl. subclasses): {Vehicle.get_total_vehicles()}") # Tracks all Vehicle-based instances
    print(f"Tesla is vintage? {ElectricCar.is_vintage(my_tesla.year)}") # Inherited static method

except ValueError as e:
     logging.error(f"Error during subclass demonstration: {e}")

2025-04-20 16:13:27,659 - INFO - Created Vehicle: Tesla Model 3 (2022)
2025-04-20 16:13:27,661 - INFO - Created ElectricCar: Tesla Model 3 with 75.0kWh battery
2025-04-20 16:13:27,663 - INFO - Created Vehicle: Ford F-150 (2021)
2025-04-20 16:13:27,664 - INFO - Created Truck: Ford F-150 with 10000lbs towing
2025-04-20 16:13:27,667 - INFO - Creating ElectricCar from string using ElectricCar
2025-04-20 16:13:27,668 - INFO - Created Vehicle: Nissan Leaf (2019)
2025-04-20 16:13:27,669 - INFO - Created ElectricCar: Nissan Leaf with 40.0kWh battery


2022 Tesla Model 3 with 5000 miles., Battery: 75.0kWh, Charge: 100.0%
2021 Ford F-150 with 12000 miles., Towing Capacity: 10000 lbs
Tesla Model 3 drove 100 miles. Total mileage: 5100
Battery level after driving: 66.7%
Tesla Model 3 charged. Current level: 86.7%
Ford F-150 is towing 8000 lbs.
Cannot tow 12000 lbs. Exceeds capacity of 10000 lbs.
Not enough charge for 5000 miles. Can drive approx 260 miles.
Tesla Model 3 drove 260 miles. Total mileage: 5360
Electric Car from string: ElectricCar(make='Nissan', model='Leaf', year=2019, battery_kwh=40.0, initial_mileage=22000)
2019 Nissan Leaf with 22000 miles., Battery: 40.0kWh, Charge: 100.0%

Total vehicles (incl. subclasses): 6
Tesla is vintage? False


**Pitfall: The Fragile Base Class Problem.** If you modify a base class (e.g., change method signatures, remove attributes), it can potentially break all subclasses that depend on the old structure. Careful design and testing are essential. Consider composition over inheritance if the "IS-A" relationship isn't strong.
**Pitfall: Overuse of Inheritance.** Deep inheritance hierarchies (e.g., A -> B -> C -> D -> E) can become complex and hard to manage. Sometimes, it's better to use **Composition** (HAS-A relationship - an object contains instances of other objects) or **Mixins** (classes providing specific functionality meant to be inherited by multiple unrelated classes).

## 5. Polymorphism: Many Forms, One Interface
**Concept:** The ability of objects of different classes to respond to the same method call in their own specific way. It allows you to treat objects of different types uniformly if they share a common interface (method name).
**Python's Approach:**
*   **Duck Typing:** "If it walks like a duck and quacks like a duck, then it must be a duck." Python doesn't strictly require an explicit interface or inheritance. If an object has the method you're calling, Python will try to execute it. This is the most common form of polymorphism in Python.
*   **Inheritance-based Polymorphism:** As seen above, when subclasses override methods from a superclass, calling that method on different subclass objects results in different behaviors.
*   **Abstract Base Classes (ABCs):** Provide a way to define formal interfaces, ensuring subclasses implement specific methods (covered next under Abstraction).
**Real-world Analogy:** Think of a USB port. You can plug in a mouse, a keyboard, a flash drive, or a webcam. They are different devices (different classes), but they all connect via the same interface (the USB port / the method call), and the computer interacts with each appropriately based on what it is.

### 5.1 Demonstrating Polymorphism (Duck Typing & Inheritance)

In [25]:
# Using the vehicles created earlier: car1 (Vehicle), my_tesla (ElectricCar), my_f150 (Truck)
vehicles: List[Vehicle] = [
    Vehicle("Generic", "Sedan", 2015, 50000),
    my_tesla,
    my_f150
]

# This function works with any object that has a 'drive' method
# and a 'get_description' method (Duck Typing in action if we didn't use inheritance)
# With inheritance, it leverages the overridden methods.
def test_drive_vehicle(vehicle: Vehicle, distance: int):
    """Tests driving a vehicle and prints its description."""
    print("-" * 30)
    print(f"Testing: {vehicle!s}") # Uses __str__
    try:
        print(f"Before drive: {vehicle.get_description()}")
        vehicle.drive(distance)
        print(f"After drive: {vehicle.get_description()}")
    except Exception as e:
        print(f"Could not drive {vehicle!s}: {e}")
    print("-" * 30)

# Iterate and call the same methods on different object types
for v in vehicles:
    test_drive_vehicle(v, 50)

2025-04-20 16:13:27,688 - INFO - Created Vehicle: Generic Sedan (2015)


------------------------------
Testing: 2015 Generic Sedan
Before drive: 2015 Generic Sedan with 50000 miles.
Generic Sedan drove 50 miles. Total mileage: 50050
After drive: 2015 Generic Sedan with 50050 miles.
------------------------------
------------------------------
Testing: 2022 Tesla Model 3
Before drive: 2022 Tesla Model 3 with 5360 miles., Battery: 75.0kWh, Charge: 0.0%
Tesla Model 3 battery is empty! Cannot drive.
After drive: 2022 Tesla Model 3 with 5360 miles., Battery: 75.0kWh, Charge: 0.0%
------------------------------
------------------------------
Testing: 2021 Ford F-150
Before drive: 2021 Ford F-150 with 12000 miles., Towing Capacity: 10000 lbs
Ford F-150 drove 50 miles. Total mileage: 12050
After drive: 2021 Ford F-150 with 12050 miles., Towing Capacity: 10000 lbs
------------------------------


## 6. Abstraction: Hiding Complexity, Defining Contracts
**Concept:** Hiding the complex implementation details of an object and exposing only the essential features or functionalities. It helps in managing complexity by focusing on *what* an object does rather than *how* it does it. Abstraction is often achieved through Abstract Base Classes (ABCs).
**Abstract Base Classes (ABCs):**
*   Defined using the `abc` module (`ABC`, `@abstractmethod`).
*   Cannot be instantiated directly.
*   Can define "abstract methods" that subclasses *must* implement. This enforces a common interface (a contract) for all subclasses.
*   Can also contain concrete methods (methods with implementation) that subclasses can inherit or override.
**Real-world Analogy:** A TV remote control. It provides an abstract interface (buttons for power, volume, channels). You don't need to know the intricate circuitry (`implementation details`) inside to use it. The remote enforces a contract: a "power" button *must* turn the TV on/off.

### 6.1 Using `abc` Module for Abstraction

In [26]:
from abc import ABC, abstractmethod

# Define an Abstract Base Class for data processing
class DataProcessor(ABC):
    """
    Abstract base class for data processing tasks.
    Defines a contract for loading and processing data.
    """

    def __init__(self, source: str):
        self.source = source
        self._data: Optional[List[str]] = None # Protected attribute for loaded data
        logging.info(f"DataProcessor initialized for source: {self.source}")

    @abstractmethod
    def load_data(self) -> None:
        """Abstract method to load data from the source."""
        # Subclasses MUST implement this
        logging.warning("load_data called on abstract class - should be overridden.")
        pass # Or raise NotImplementedError

    @abstractmethod
    def process_data(self) -> int:
        """
        Abstract method to process the loaded data.
        Returns the number of records processed.
        """
        # Subclasses MUST implement this
        logging.warning("process_data called on abstract class - should be overridden.")
        if self._data is None:
             raise ValueError("Data not loaded before processing.")
        return 0 # Placeholder

    # Concrete method available to all subclasses
    def run_pipeline(self) -> int:
        """Loads and processes data, returning the record count."""
        print(f"--- Running pipeline for {type(self).__name__} from {self.source} ---")
        try:
            self.load_data()
            if self._data is not None:
                print(f"Data loaded successfully ({len(self._data)} lines).")
                processed_count = self.process_data()
                print(f"Processing complete. {processed_count} records processed.")
                print("-" * (len(self.source) + 40))
                return processed_count
            else:
                print("Data loading failed or yielded no data.")
                print("-" * (len(self.source) + 40))
                return 0
        except Exception as e:
            logging.error(f"Pipeline failed for {self.source}: {e}")
            print(f"Error during pipeline execution: {e}")
            print("-" * (len(self.source) + 40))
            return -1 # Indicate error


# Concrete implementation for CSV files
class CsvDataProcessor(DataProcessor):
    """Processes data from a CSV file."""

    def load_data(self) -> None:
        """Loads data from a CSV file (simulated)."""
        print(f"Simulating loading data from CSV: {self.source}")
        # In a real scenario, use pandas or csv module
        # Simulating data loading
        try:
            # Pretend we read lines from a file
            # In a real case: with open(self.source, 'r') as f: self._data = f.readlines()
            if "missing" in self.source: raise FileNotFoundError("Simulated missing file")
            self._data = [
                "header1,header2,header3",
                "data1,10,True",
                "data2,25,False",
                "data3,5,True"
            ]
            logging.info(f"Loaded {len(self._data)} lines from simulated CSV {self.source}")
        except FileNotFoundError as e:
            logging.error(f"CSV file not found: {self.source}. Error: {e}")
            self._data = None # Ensure data is None on failure
            raise # Re-raise the exception to be caught by run_pipeline

    def process_data(self) -> int:
        """Processes the loaded CSV data (e.g., count non-header rows)."""
        if self._data is None:
            print("No data loaded for processing.")
            return 0
        print("Processing CSV data (counting non-header rows)...")
        # Simple processing: count lines excluding header
        count = len(self._data) - 1 if len(self._data) > 0 else 0
        logging.info(f"Processed {count} data rows from {self.source}")
        return max(0, count) # Ensure non-negative

# Concrete implementation for JSON data (perhaps from an API)
class JsonApiProcessor(DataProcessor):
    """Processes data fetched from a JSON API endpoint."""

    def load_data(self) -> None:
        """Loads data from a JSON API (simulated)."""
        print(f"Simulating fetching data from JSON API: {self.source}")
        # In a real scenario, use requests library and json module
        # Simulating API call
        import json
        try:
             # Pretend we get a JSON response
             # In real case: response = requests.get(self.source); response.raise_for_status(); api_data = response.json()
             if "error" in self.source: raise ConnectionError("Simulated API connection error")
             api_data = [
                {'id': 1, 'value': 'A'},
                {'id': 2, 'value': 'B'}
             ]
             # Store data as list of strings for consistency with base class example
             self._data = [json.dumps(item) for item in api_data]
             logging.info(f"Loaded {len(self._data)} records from simulated API {self.source}")
        except ConnectionError as e:
             logging.error(f"API connection failed: {self.source}. Error: {e}")
             self._data = None
             raise

    def process_data(self) -> int:
        """Processes the loaded JSON data (e.g., count records)."""
        if self._data is None:
            print("No data loaded for processing.")
            return 0
        print("Processing JSON data (counting records)...")
        count = len(self._data)
        logging.info(f"Processed {count} JSON records from {self.source}")
        return count

In [27]:
# Try to instantiate the Abstract Base Class (will fail)
try:
    # processor = DataProcessor("some_source") # This will raise TypeError
    print("Successfully created DataProcessor instance (This shouldn't happen!)")
except TypeError as e:
    print(f"As expected, cannot instantiate abstract class: {e}")

# Create instances of concrete subclasses
csv_processor = CsvDataProcessor("data/my_data.csv")
api_processor = JsonApiProcessor("https://api.example.com/data")
csv_processor_missing = CsvDataProcessor("data/missing_file.csv")
api_processor_error = JsonApiProcessor("https://api.example.com/error")

# Polymorphism in action using the common interface defined by the ABC
processors: List[DataProcessor] = [csv_processor, api_processor, csv_processor_missing, api_processor_error]

results = {}
for processor in processors:
    # The run_pipeline method works uniformly because the contract
    # (load_data, process_data) is enforced by the ABC.
    result_count = processor.run_pipeline()
    results[processor.source] = result_count

print("\n--- Pipeline Results ---")
print(results)

2025-04-20 16:13:27,730 - INFO - DataProcessor initialized for source: data/my_data.csv


Successfully created DataProcessor instance (This shouldn't happen!)


2025-04-20 16:13:27,733 - INFO - DataProcessor initialized for source: https://api.example.com/data
2025-04-20 16:13:27,735 - INFO - DataProcessor initialized for source: data/missing_file.csv
2025-04-20 16:13:27,736 - INFO - DataProcessor initialized for source: https://api.example.com/error
2025-04-20 16:13:27,738 - INFO - Loaded 4 lines from simulated CSV data/my_data.csv
2025-04-20 16:13:27,739 - INFO - Processed 3 data rows from data/my_data.csv


--- Running pipeline for CsvDataProcessor from data/my_data.csv ---
Simulating loading data from CSV: data/my_data.csv
Data loaded successfully (4 lines).
Processing CSV data (counting non-header rows)...


2025-04-20 16:13:27,741 - INFO - Loaded 2 records from simulated API https://api.example.com/data
2025-04-20 16:13:27,743 - INFO - Processed 2 JSON records from https://api.example.com/data
2025-04-20 16:13:27,743 - ERROR - CSV file not found: data/missing_file.csv. Error: Simulated missing file
2025-04-20 16:13:27,744 - ERROR - Pipeline failed for data/missing_file.csv: Simulated missing file
2025-04-20 16:13:27,746 - ERROR - API connection failed: https://api.example.com/error. Error: Simulated API connection error
2025-04-20 16:13:27,746 - ERROR - Pipeline failed for https://api.example.com/error: Simulated API connection error


Processing complete. 3 records processed.
--------------------------------------------------------
--- Running pipeline for JsonApiProcessor from https://api.example.com/data ---
Simulating fetching data from JSON API: https://api.example.com/data
Data loaded successfully (2 lines).
Processing JSON data (counting records)...
Processing complete. 2 records processed.
--------------------------------------------------------------------
--- Running pipeline for CsvDataProcessor from data/missing_file.csv ---
Simulating loading data from CSV: data/missing_file.csv
Error during pipeline execution: Simulated missing file
-------------------------------------------------------------
--- Running pipeline for JsonApiProcessor from https://api.example.com/error ---
Simulating fetching data from JSON API: https://api.example.com/error
Error during pipeline execution: Simulated API connection error
---------------------------------------------------------------------

--- Pipeline Results ---
{'da

## 7. Enterprise Considerations & Best Practices
Building robust, maintainable software requires more than just understanding OOP principles. Here are some related best practices crucial in enterprise environments:

### 7.1 Modularity and Design Patterns
*   **Modularity:** Break down large systems into smaller, independent modules (Python files/packages). Classes are fundamental building blocks for modules.
*   **Design Patterns:** Proven solutions to common software design problems (e.g., Factory, Singleton, Strategy, Observer). OOP makes implementing many patterns more natural.
*   **SOLID Principles:** Guidelines for creating understandable, flexible, and maintainable OOP designs.
    *   **S**ingle Responsibility Principle: A class should have only one reason to change.
    *   **O**pen/Closed Principle: Software entities (classes, modules) should be open for extension but closed for modification. (Inheritance/Composition help here).
    *   **L**iskov Substitution Principle: Subtypes must be substitutable for their base types without altering the correctness of the program. (Important for inheritance).
    *   **I**nterface Segregation Principle: Clients should not be forced to depend on interfaces they do not use. (Smaller, more specific interfaces/ABCs are better).
    *   **D**ependency Inversion Principle: High-level modules should not depend on low-level modules. Both should depend on abstractions. Abstractions should not depend on details. (ABCs and Dependency Injection help).

### 7.2 Dependency Injection (DI)
**Concept:** Instead of a class creating its own dependencies (objects it needs to function), these dependencies are "injected" from the outside (usually via the constructor).
**Benefits:**
*   **Decoupling:** Classes are less tied to specific implementations of their dependencies.
*   **Testability:** Easier to provide mock or fake dependencies during testing.
*   **Flexibility:** Easier to swap out different implementations of a dependency.
**Simple Example:**

In [28]:
from typing import Protocol # Using Protocol for structural typing / interface definition

# Define an interface (Protocol) for logging
class Logger(Protocol):
    def log(self, message: str) -> None:
        ...

# Concrete implementation using standard logging
class StandardLogger:
    def log(self, message: str) -> None:
        logging.info(f"[StandardLogger] {message}")

# Another implementation (e.g., prints to console)
class ConsoleLogger:
     def log(self, message: str) -> None:
        print(f"[ConsoleLogger] {message}")

# A service class that *depends* on a Logger
class NotificationService:
    """Sends notifications using an injected logger."""

    # Dependency is injected via the constructor
    def __init__(self, logger: Logger):
        self._logger = logger # Store the injected dependency

    def send_notification(self, user: str, message: str) -> None:
        """Simulates sending a notification."""
        log_message = f"Sending notification to {user}: '{message}'"
        print(log_message) # Simulate actual sending
        # Use the injected logger
        self._logger.log(log_message)

# --- How to use DI ---

# Create logger instances (dependencies)
standard_logger = StandardLogger()
console_logger = ConsoleLogger()

# Inject the desired logger into the service
service1 = NotificationService(logger=standard_logger)
service2 = NotificationService(logger=console_logger)

# Use the services - they behave the same way but log differently
service1.send_notification("Alice", "Your order has shipped!")
service2.send_notification("Bob", "Password reset requested.")

2025-04-20 16:13:27,766 - INFO - [StandardLogger] Sending notification to Alice: 'Your order has shipped!'


Sending notification to Alice: 'Your order has shipped!'
Sending notification to Bob: 'Password reset requested.'
[ConsoleLogger] Sending notification to Bob: 'Password reset requested.'


**Enterprise Note:** In large applications, Dependency Injection Containers (Frameworks like `python-dependency-injector`, or built-in features of web frameworks like FastAPI, Django) manage the creation and injection of dependencies automatically.

### 7.3 Logging
*   **Importance:** Essential for monitoring application behavior, diagnosing issues in development and production.
*   **Best Practice:** Use Python's built-in `logging` module. Avoid `print()` statements for operational logging. Configure log levels (DEBUG, INFO, WARNING, ERROR, CRITICAL) appropriately.
*   **(Self-Correction):** We've already incorporated basic logging using `logging.basicConfig` and calls like `logging.info`, `logging.error`. In enterprise apps, configuration is usually more complex (e.g., logging to files, external services, rotating logs).

### 7.4 Testing (Unit Testing)
*   **Importance:** Ensures code correctness, prevents regressions, and facilitates refactoring.
*   **Best Practice:** Use frameworks like `unittest` (built-in) or `pytest`. Write tests for individual classes and methods (unit tests).
*   **OOP & Testing:** Encapsulation and Dependency Injection make classes easier to test in isolation.

In [29]:
import unittest

# Basic test suite for our Vehicle class
class TestVehicle(unittest.TestCase):

    def test_vehicle_creation(self):
        """Test basic vehicle instantiation and attributes."""
        v = Vehicle("TestMake", "TestModel", 2023, 100)
        self.assertEqual(v.make, "TestMake")
        self.assertEqual(v.model, "TestModel")
        self.assertEqual(v.year, 2023)
        self.assertEqual(v._mileage, 100) # Test protected attribute access within tests

    def test_drive_method(self):
        """Test the drive method increases mileage."""
        v = Vehicle("TestMake", "TestModel", 2023, 100)
        v.drive(50)
        self.assertEqual(v._mileage, 150)

    def test_get_description(self):
        """Test the description string format."""
        v = Vehicle("TestMake", "TestModel", 2023, 100)
        self.assertEqual(v.get_description(), "2023 TestMake TestModel with 100 miles.")

    def test_invalid_year(self):
        """Test that creating a vehicle with an invalid year raises ValueError."""
        with self.assertRaises(ValueError):
            Vehicle("TestMake", "TestModel", 1800)

    def test_invalid_mileage(self):
        """Test that creating a vehicle with negative mileage raises ValueError."""
        with self.assertRaises(ValueError):
            Vehicle("TestMake", "TestModel", 2023, -50)

    def test_drive_negative_distance(self):
        """Test driving negative distance raises ValueError."""
        v = Vehicle("TestMake", "TestModel", 2023)
        with self.assertRaises(ValueError):
            v.drive(-10)

    @classmethod
    def setUpClass(cls):
        # Reset class attribute for predictable test runs if needed
        Vehicle.num_vehicles_created = 0

# --- Running Tests within Jupyter (simplified) ---
# Usually, you run tests from the command line using 'python -m unittest your_test_module.py'
# Or 'pytest'

# This creates a test suite and runs it, capturing the output.
suite = unittest.TestLoader().loadTestsFromTestCase(TestVehicle)
runner = unittest.TextTestRunner()
print("--- Running Vehicle Unit Tests ---")
result = runner.run(suite)
print(f"--- Tests Run: {result.testsRun}, Failures: {len(result.failures)}, Errors: {len(result.errors)} ---")

2025-04-20 16:13:27,789 - INFO - Created Vehicle: TestMake TestModel (2023)
.2025-04-20 16:13:27,792 - INFO - Created Vehicle: TestMake TestModel (2023)
.2025-04-20 16:13:27,794 - INFO - Created Vehicle: TestMake TestModel (2023)
.

--- Running Vehicle Unit Tests ---
TestMake TestModel drove 50 miles. Total mileage: 150


..2025-04-20 16:13:27,800 - INFO - Created Vehicle: TestMake TestModel (2023)
.
----------------------------------------------------------------------
Ran 6 tests in 0.013s

OK


--- Tests Run: 6, Failures: 0, Errors: 0 ---


### 7.5 Type Hinting, Docstrings, PEP 8
*   **Type Hinting (`typing` module):** Improves code readability, allows static analysis tools (`mypy`) to catch type errors early. Crucial for large codebases. (Used throughout this notebook).
*   **Docstrings:** Explain what classes, methods, and functions do. Essential for documentation generation and maintainability. (Used throughout this notebook).
*   **PEP 8:** The official Python style guide. Ensures code consistency and readability across projects and teams. Use linters (`flake8`, `pylint`) and formatters (`black`, `isort`). (Adhered to in this notebook).

## 8. Pitfalls and Common Interview Questions
**Common Pitfalls:**
1.  **Mutable Default Arguments:** Using mutable types (lists, dicts) as default values in `__init__` or methods. The default is shared across calls/instances, leading to unexpected behavior.
    ```python
    # Bad example:
    # def __init__(self, name, items=[]): self.items = items # items will be shared!
    # Correct:
    # def __init__(self, name, items=None): self.items = items if items is not None else []
    ```
2.  **Misunderstanding `super()`:** Incorrectly calling or omitting `super().__init__()` in subclasses, especially in multiple inheritance scenarios (MRO - Method Resolution Order).
3.  **Confusing `@classmethod` and `@staticmethod`:** Using one when the other is appropriate. Remember: `@classmethod` gets `cls`, `@staticmethod` gets neither `self` nor `cls`.
4.  **Overriding vs Overloading:** Python doesn't support traditional method overloading (multiple methods with the same name but different parameter types) like Java/C++. It uses overriding (subclass provides its own implementation of a parent method). Default arguments or `*args/**kwargs` achieve flexibility.
5.  **Ignoring Encapsulation:** Directly accessing `_protected` or `__private` attributes from outside the class unnecessarily.
6.  **Choosing Inheritance when Composition is Better:** Forcing an "IS-A" relationship when a "HAS-A" relationship is more suitable and flexible.
**Common OOP Interview Questions:**
1.  Explain the four main pillars of OOP (Encapsulation, Abstraction, Inheritance, Polymorphism) with examples.
2.  What is a class? What is an object?
3.  Explain `__init__`, `self`.
4.  What is inheritance? What are its benefits and drawbacks?
5.  What is `super()` used for?
6.  Explain polymorphism in Python. What is Duck Typing?
7.  What is encapsulation? How is it achieved in Python (`_`, `__`, properties)?
8.  What is abstraction? How do Abstract Base Classes (ABCs) work?
9.  What is the difference between `@classmethod` and `@staticmethod`? When would you use each?
10. What are `__str__` and `__repr__` used for?
11. What is the difference between Composition and Inheritance? When prefer one over the other?
12. What are SOLID principles? (Explain one or two).
13. What is Dependency Injection? Why is it useful?

## 9. Mini-Project Challenge: Simple Library Management System
**Goal:** Apply the OOP concepts learned to model a basic library system.
**Requirements:**
1.  **`Book` Class:**
    *   Attributes: `title` (str), `author` (str), `isbn` (str, make it "private" or "protected"), `is_checked_out` (bool, default False).
    *   Methods:
        *   `__init__` to initialize attributes.
        *   `check_out()`: Sets `is_checked_out` to `True`. Log this action.
        *   `check_in()`: Sets `is_checked_out` to `False`. Log this action.
        *   `__str__`: Returns "Title by Author".
        *   `__repr__`: Returns `Book(title='...', author='...', isbn='...')`.
        *   A property `isbn` to get the ISBN (read-only access from outside).
2.  **`Member` Class:**
    *   Attributes: `name` (str), `member_id` (str, "private"/"protected"), `checked_out_books` (List[Book], initialize empty).
    *   Methods:
        *   `__init__`
        *   `borrow_book(book: Book)`: Adds the book to `checked_out_books` *if* it's not already checked out. Calls the book's `check_out()` method. Returns `True` if successful, `False` otherwise. Log success/failure.
        *   `return_book(book: Book)`: Removes the book from `checked_out_books`. Calls the book's `check_in()` method. Log the action.
        *   `__str__`: Returns "Member: Name (ID: member_id)".
        *   `__repr__`.
        *   A property `member_id` (read-only).
3.  **`Library` Class:**
    *   Attributes: `name` (str), `catalog` (Dict[str, Book], mapping ISBN to Book object), `members` (Dict[str, Member], mapping member ID to Member object). Use Dependency Injection for a `Logger` instance.
    *   Methods:
        *   `__init__(name: str, logger: Logger)`
        *   `add_book(book: Book)`: Adds a book to the catalog. Log the action.
        *   `add_member(member: Member)`: Adds a member. Log the action.
        *   `find_book(isbn: str) -> Optional[Book]`: Returns the book or None.
        *   `find_member(member_id: str) -> Optional[Member]`: Returns the member or None.
        *   `lend_book(member_id: str, isbn: str) -> bool`: Finds member and book. If both exist and book is available, calls the member's `borrow_book` method. Log the attempt and result. Returns `True` on success.
        *   `accept_return(member_id: str, isbn: str) -> bool`: Finds member and book. If both exist and the member has the book checked out, calls the member's `return_book` method. Log the attempt and result. Returns `True` on success.
**Bonus:**
*   Add different types of library items (e.g., `DVD`, `Journal`) using inheritance from a base `LibraryItem` ABC.
*   Implement basic error handling (e.g., book not found, member already has book).
*   Write simple `unittest` cases for your classes.

```python
# --- Solution Space for Mini-Project ---
from typing import List, Dict, Optional
import uuid # For generating unique IDs
# Assuming Logger protocol and StandardLogger/ConsoleLogger are defined as above
# If not, redefine them here or import them
# (Implement Book, Member, Library classes here)
# --- Example Usage ---
# my_logger = StandardLogger()
# my_library = Library("City Central Library", logger=my_logger)
#
# book1 = Book("The Hitchhiker's Guide", "Douglas Adams", "978-0345391803")
# book2 = Book("Pride and Prejudice", "Jane Austen", "978-0141439518")
# member1 = Member("Alice Wonderland", str(uuid.uuid4()))
#
# my_library.add_book(book1)
# my_library.add_book(book2)
# my_library.add_member(member1)
#
# my_library.lend_book(member1.member_id, book1.isbn)
# my_library.lend_book(member1.member_id, book1.isbn) # Try lending again (should fail)
# my_library.accept_return(member1.member_id, book1.isbn)
```

## 10. Conclusion
Object-Oriented Programming is a powerful paradigm for structuring code, especially for larger, complex applications. By mastering Classes, Inheritance, Encapsulation, Abstraction, and Polymorphism, and combining them with best practices like logging, testing, type hinting, and dependency injection, you can write Python code that is:
*   **Readable and Understandable**
*   **Maintainable and Extensible**
*   **Reusable and Modular**
*   **Robust and Testable**
Continue practicing these concepts, explore design patterns, and strive to write clean, well-structured code. Happy coding!

**Further Learning:**
*   Python Official Documentation (`abc`, `typing`, `unittest`, `logging` modules)
*   Books: "Fluent Python" by Luciano Ramalho, "Python Cookbook" by David Beazley & Brian K. Jones
*   Online Courses and Tutorials on Python OOP and Design Patterns.
*   Explore `attrs` or `dataclasses` for boilerplate reduction in classes.
*   Investigate advanced topics like Metaclasses and Descriptors.